<a href="https://colab.research.google.com/github/Decoding-Data-Science/nov25/blob/main/Weather_LLM_wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install openai==0.28 requests

In [ ]:
# import os
# from google.colab import userdata

import openai
# Retrieve API keys from Colab's secure storage
from dotenv import load_dotenv

API_KEY = os.getenv("OPENAI_API_KEY")

openai_api_key = API_KEY 
# Set them as environment variables

# if openai_api_key:
#     os.environ["OPENAI_API_KEY"] = openai_api_key

In [2]:
import requests

def get_current_weather(location, unit='celsius'):
    weather_api_key = "a7b109315b6ced6a9cd5177bdb98ca82"
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(base_url)
    data = response.json()

    weather_description = data['weather'][0]['description']
    return {
        "location": location,
        "temperature": data['main']['temp'],
        "weather": weather_description
    }

In [3]:
print(get_current_weather("Dubai"))

{'location': 'Dubai', 'temperature': 26.96, 'weather': 'broken clouds'}


In [4]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city, e.g. San Francisco"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

In [6]:
functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city, e.g. San Francisco'},
    'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}},
   'required': ['location']}}]

In [ ]:
import json
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
# better: read from env
client = OpenAI(api_key=API_KEY)

# Define tools (old "functions")
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather for a given city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name, e.g. 'Dubai'"
                    }
                },
                "required": ["location"]
            },
        },
    }
]

def get_current_weather(location: str):
    """
    Your own function that actually fetches weather.
    For now you can dummy it, e.g.:
    """
    # TODO: replace with real API call
    return {
        "location": location,
        "temperature_c": 38,
        "condition": "Sunny"
    }

def weather_chat(user_message: str) -> str:
    # System message should be role="system", not "assistant"
    messages = [
        {
            "role": "system",
            "content": "You are a weather bot. Answer only in Celsius. and only get cities tempretures and dont answer anything else"
        },
        {
            "role": "user",
            "content": user_message
        }
    ]

    # 1) First call: let the model decide whether to call the tool
    first_response = client.chat.completions.create(
        model="gpt-4o-mini",            # or "gpt-4.1-mini","gpt-3.5-turbo-0125", etc.
        messages=messages,
        tools=tools,                    # old: functions=functions
        tool_choice="auto",
        temperature=0,
        max_tokens=256,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
    )

    msg = first_response.choices[0].message

    # If the model didn't call a tool, just answer directly
    if not msg.tool_calls:
        return msg.content or "I'm here to provide weather updates."

    # 2) Handle the tool call
    tool_call = msg.tool_calls[0]  # assuming only one call
    tool_name = tool_call.function.name
    tool_args = json.loads(tool_call.function.arguments)

    if tool_name != "get_current_weather" or "location" not in tool_args:
        return "I'm here to provide weather updates. Please ask me questions related to weather."

    # Call your Python function
    weather_data = get_current_weather(tool_args["location"])

    # 3) Add assistant tool call + tool result to the conversation
    messages.append({
        "role": "assistant",
        "tool_calls": [
            {
                "id": tool_call.id,
                "type": "function",
                "function": {
                    "name": tool_name,
                    "arguments": tool_call.function.arguments,
                }
            }
        ]
    })

    messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": tool_name,
        "content": json.dumps(weather_data),
    })

    # 4) Second call: let the model convert tool output into a nice answer
    second_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        max_tokens=256
    )

    return second_response.choices[0].message.content


In [7]:

weather_chat("who is the presedint of Dubai")

"I'm only able to provide weather information for cities. Please let me know which city's temperature you would like to know."

In [12]:
#qc
weather_chat("كم درجة الحرارة في الشارقة؟ ")

'درجة الحرارة في الشارقة هي 38 درجة مئوية.'

In [ ]:
# !pip install gradio

In [13]:
# Define Gradio interface
import gradio as gr
iface = gr.Interface(
    fn=weather_chat,
    inputs=gr.Textbox(label="Weather Queries"),
    outputs=gr.Textbox(label="Weather Updates"),
    title = "Weather Bot",
    description = "Ask me anything about weather!"
)

# Launch the Gradio interface
iface.launch(share="True")

c:\Users\almehairbi\Desktop\Dell AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://c711d84217a28c9637.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
